In [ ]:
import cartopy.io.shapereader as shpreader
import matplotlib.pyplot as plt
import geopandas as gpd
import numpy as np
import pandas as pd
import os
import sys
from osgeo import gdal,ogr, osr

In [ ]:
## mosaic land cover data
path = "../1_original_chunks/"

file_list1 = os.listdir(f"{path}{2020}")
file_list2 = os.listdir(f"{path}{2021}")
file_list3 = os.listdir(f"{path}{2022}")

d1 = [f"{path}2020/{x}" for x in file_list1]
d2 = [f"{path}2021/{x}" for x in file_list2]
d3 = [f"{path}2022/{x}" for x in file_list3]

out1 = "../2_mosaic_chunks/2020_US_land_cover.tif"
out2 = "../2_mosaic_chunks/2021_US_land_cover.tif"
out3 = "../2_mosaic_chunks/2022_US_land_cover.tif"
options = gdal.WarpOptions(format='GTiff')

mosaic = gdal.Warp(out1, d1, options=options)
mosaic = None 
print("Mosaic created successfully at:", out1)

mosaic = gdal.Warp(out2, d2, options=options)
mosaic = None 
print("Mosaic created successfully at:", out2)

mosaic = gdal.Warp(out3, d3, options=options)
mosaic = None 
print("Mosaic created successfully at:", out3)

In [ ]:
## clip land cover data
years = [2020,2021,2022]
for year in years:
    input_geotiff = f"../2_mosaic_chunks/{year}_US_land_cover.tif"
    shapefile = "../3_clipped_us_land_cover/us_shapefile/us.shp"
    output_geotiff = f"../3_clipped_us_land_cover/{year}_US_land_cover_clipped.tif"
    
    input_ds = gdal.Open(input_geotiff)
    output_ds = gdal.Warp(output_geotiff, input_ds, cutlineDSName=shapefile, cropToCutline=True, dstNodata=0)
    
    input_ds = None
    output_ds = None
    shape_ds = None
    print(year,"finished")

In [ ]:
## clip land cover data
years = [2020,2021,2022]
for year in years:
    input_geotiff = f"../3_clipped_us_land_cover/{year}_US_land_cover_clipped.tif"
    shapefile = "../3_clipped_us_land_cover/us_shapefile/us_extent.shp"
    output_geotiff = f"../3_clipped_us_land_cover/{year}_US_land_cover_clip.tif"
    
    input_ds = gdal.Open(input_geotiff)
    output_ds = gdal.Warp(output_geotiff, input_ds, cutlineDSName=shapefile, cropToCutline=True, dstNodata=0)
    input_ds = None
    output_ds = None
    shape_ds = None
    print(year,"finished")
    os.remove(input_geotiff)

In [ ]:
## clip using NEON site extent
def read_tif(tif_file):
    dataset = gdal.Open(tif_file)
    cols = dataset.RasterXSize
    rows = dataset.RasterYSize
    im_proj = (dataset.GetProjection())
    im_Geotrans = (dataset.GetGeoTransform())
    im_data = np.moveaxis(dataset.ReadAsArray(0, 0, cols, rows), 0, -1)
    return im_data, im_Geotrans, im_proj,rows, cols

prisma_path = "/Volumes/ChenLab/Fujiang/0_Seasonal_PRISMA_traits/2_PRISMA_L2D/7_PRISMA_latlon/"
land_use_path = "/Volumes/ChenLab/Fujiang/0_Seasonal_PRISMA_traits/3_GLC_FCS30D_Land_cover_data_US/3_clipped_us_land_cover/"
out_path = "/Volumes/ChenLab/Fujiang/0_Seasonal_PRISMA_traits/3_GLC_FCS30D_Land_cover_data_US/4_land_cover_NEON_sites/"

years = [2020,2021,2022]
site_list = os.listdir(prisma_path)
for site in site_list:
    path = f"{prisma_path}{site}"
    file_list = os.listdir(path)
    file_list = [x for x in file_list if "LATLON.tif" in x]
    os.makedirs(f"{out_path}{site}", exist_ok=True)
    
    ul_x_all,lr_y_all, lr_x_all, ul_y_all = [],[],[],[]
    for file in file_list:
        im_data, im_Geotrans, im_proj,rows, cols = read_tif(f"{path}/{file}")
        ul_x, lr_y, lr_x, ul_y = im_data[0,0,1],im_data[-1,-1,0],im_data[-1,-1,1],im_data[0,0,0]
        ul_x_all.append(ul_x)
        lr_y_all.append(lr_y)
        lr_x_all.append(lr_x)
        ul_y_all.append(ul_y)
    ul_x, lr_y, lr_x, ul_y = min(ul_x_all),min(lr_y_all), max(lr_x_all), max(ul_y_all)
    ul_x, lr_y, lr_x, ul_y = ul_x-0.2, lr_y-0.2, lr_x+0.2, ul_y+0.2
    for year in years:
        input_tif = f"{land_use_path}{year}_US_land_cover_clip.tif"
        output_tif = f"{out_path}{site}/{site}_{year}_land_cover.tif"
        gdal.Warp(output_tif, input_tif, format = 'GTiff', outputBounds=(ul_x, lr_y, lr_x, ul_y))
        input_tif = None
        output_tif = None
        print(site,year,"finished")
        
        ### transfer the projection of clipped land use data
        in_tif = f"{out_path}{site}/{site}_{year}_land_cover.tif"
        out_tif = f"{out_path}{site}/{site}_{year}_land_cover.tif"
        input_ds = gdal.Open(in_tif)
        output_ds = gdal.Warp(out_tif, input_ds, dstSRS=im_proj)